In [1]:
from xarm.wrapper import XArmAPI
import numpy as np
import time, csv, threading

# Connect to the robot
arm = XArmAPI('192.168.1.225', is_radian=True)

arm.motion_enable(enable=True)
arm.set_mode(0)    # Position control mode
arm.set_state(0)

CENTER = np.array([470.0, 0.0])
R = 40.0
Z = 310
RPY = (180, 0, 0)
SLOW = 50
FAST = 75

SDK_VERSION: 1.17.0
ROBOT_IP: 192.168.1.225, VERSION: v2.4.0, PROTOCOL: V1, DETAIL: 7,7,XS1304,AC1303,v2.4.0, TYPE1300: [1, 1]
change protocol identifier to 3


In [2]:
def move_home():
    # Move to home position
    arm.set_position(x=470, y=0, z=385, roll=180, pitch=0, yaw=0, speed=SLOW, is_radian=False)

def move_center():
    # Move to center position
    arm.set_position(x=470, y=0, z=310, roll=180, pitch=0, yaw=0, speed=SLOW, is_radian=False)

def wrap_up():
    move_center()
    move_home()
    arm.disconnect()

log_rows = []                      # [j0, j1, j2, j3, j4, j5, j6]
stop_evt = threading.Event()

def on_report(data):
    ang = data.get('joints')
    if ang:
        log_rows.append(ang)


In [3]:
def pt(angle_deg):
    a = np.deg2rad(angle_deg)
    x = CENTER[0] + R*np.cos(a)
    y = CENTER[1] + R*np.sin(a)
    return [x, y, Z, *RPY]

In [4]:
move_home()
# move_center()

In [ ]:
N = 6
speed, acc = FAST, 2000  # tune as needed

arm.register_report_callback(on_report)

arm.set_position(*pt(0), speed=FAST, wait=True, is_radian=False)
arm.move_circle(pt(90), pt(180), percent=100*N, speed=speed, mvacc=acc, wait=True, is_radian=False)

arm.release_report_callback(on_report)

In [ ]:
with open('6_1.csv', 'w', newline='') as f:
    w = csv.writer(f)
    w.writerow(['j0','j1','j2','j3','j4','j5','j6'])
    w.writerows(log_rows)

In [ ]:
wrap_up()